# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Problema 2
#### Aluno: Eduardo Cesar Cauduro Coelho
#### Matrícula: 202310175
#### Aluno: Felipe Geraldo de Oliveira
#### Matrícula: 202310174

Seu código deve ser testado na seguinte instância: [link](https://drive.google.com/file/d/12CeZEow-6vVgJFgzXMo0gbjV5hLCM6Zi/view?usp=sharing). O README se encontra em: [link](https://drive.google.com/file/d/1LBTdkDoTQRxUJsKLI4Z38-Uc8oUPCZQ0/view?usp=sharing).

##Modelagem do problema da mochila:

###função objetivo:
* $max \displaystyle\sum_{i=1}^n v_ix_i$

###sujeita a:
* $\sum_{i=1}^n p_ix_i ≤ c$
* $x_i \in \{0, 1\}, ∀i $

###legenda:
* $x_i$: váriavel binária do item $i$
* $p_i$: peso do item $i$
* $v_i$: valor do item $i$
* $c$: capacidade da mochila



## Criaremos a classe KnapsackModel, que resolve um problema da mochila binária:

### O código está comentado para facilitar o entendimento:

In [1]:
import heapq

class KnapsackModel():

  def __init__ (self, valores, pesos, capacidade):
    self.__capacidade = capacidade
    self.__objetivo = 0.0
    self.__modelo = {}
    for i in range(len(pesos)):
      self.__modelo[f'x{i+1}'] = {'Valor': valores[i], 'Custo': pesos[i]}
    self.__nos_explorados = 0

    self.__variaveis_ordenadas = {}

    for variavel in self.__modelo:
      self.__variaveis_ordenadas[variavel] = self.__modelo[variavel]['Valor'] / self.__modelo[variavel]['Custo']

    self.__variaveis_ordenadas = dict(sorted(self.__variaveis_ordenadas.items(), key=lambda item: item[1], reverse=True)) # ordena as variaveis com base no seu "custo-beneficio", isto é, seu valor dividido pelo seu peso

  def solve(self):
    noh_inicial =  KnapsackProblemNode(False, self.__modelo, self.__capacidade, self.__objetivo, self.__variaveis_ordenadas)
    noh_inicial.setLimites()

    heap = [noh_inicial]
    heapq.heapify(heap) #o heap ordena com base no maior limitante superior, por causa do método __lt__ da classe KnapsackProblemNode

    while heap:

      noh = heapq.heappop(heap)

      limite_sup = noh.getLimiteSuperior()
      limite_inf = noh.getLimiteInferior()

      self.__nos_explorados += 1

      if limite_sup == False: #corte por infactibilidade
        continue

      var_frac = noh.getVariavelFrac()

      if var_frac is None:  #corte por otimalidade.
        noh.setGapOtimalidade()
        self.__nos_explorados += len(heap)
        return noh #a primeira solução encontrada é a ótima, pois estamos utilizando uma max-heap. os outros nós na heap tem limite superior menor ou igual

      nohEsq = KnapsackProblemNode(False, noh.getModelo(), noh.getCapacidade(), noh.getObjetivo(),
                                   noh.getListaOrdenada(), var_frac, noh.getVariaveisAtivadas())
      nohDir = KnapsackProblemNode(True, noh.getModelo(), noh.getCapacidade(), noh.getObjetivo(),
                                   noh.getListaOrdenada(), var_frac, noh.getVariaveisAtivadas())

      nohEsq.setLimites()
      nohDir.setLimites()

      heapq.heappush(heap, nohEsq)
      heapq.heappush(heap, nohDir)

  def getNosExplorados(self):
    return self.__nos_explorados



##Em seguida, criaremos a classe KnapsackProblemNode, que representa um subproblema do modelo.

###Esta classe possui os métodos `relaxacao()` e `heuristica()`:
* O método `relaxacao()` calcula o limite superior de cada subproblema ativando as variáveis de maior custo benefício e, caso reste capacidade, a próxima variável a ser escolhida é selecionada parcialmente de modo que a mochila seja preenchida totalmente.
* O método `heuristica()` calcula o limite inferior de cada subproblema de forma semelhante ao anterior, mas não admite nenhuma variável fracionária. Caso reste capacidade sem ser possível escolher outro item inteiro, nenhum outro item é escolhido e a mochila não é preenchida totalmente.

In [2]:
class  KnapsackProblemNode():

  #atividade = true se a variavel fracionada é setada para 1 e false se setada para 0
  #capacidade = capacidade restante da mochila apos forçar variaveis a ser 1
  #objetivo = funcao objetivo apos forçar variaveis a ser 1
  #lista_ordenada = lista ordenada de variaveis que nao foram setadas por custo beneficio (valor/peso)
  #variavel = variavel fracionaria setada a ser 0 ou 1
  #variaveis_ativadas = variaveis setadas a ser 1

  def __init__ (self, ativada, modelo, capacidade, objetivo, lista_ordenada,
                variavel = None, variaveis_ativadas = None):

    if variaveis_ativadas is None:
      self.__variaveis_ativadas = []
    else:
      self.__variaveis_ativadas = variaveis_ativadas

    if variavel is not None:

      if ativada:
        objetivo += modelo[variavel]['Valor']
        capacidade -= modelo[variavel]['Custo'] # se capacidade < 0, temos infactibilidade
        self.__variaveis_ativadas.append(variavel)

      del modelo[variavel]
      del lista_ordenada[variavel]

    self.__modelo = modelo
    self.__capacidade = capacidade
    self.__objetivo = objetivo
    self.__lista_ordenada = lista_ordenada

  def getVariaveisAtivadas(self):
    return self.__variaveis_ativadas.copy()

  def getModelo(self):
    return self.__modelo.copy()

  def getCapacidade(self):
    return self.__capacidade

  def getObjetivo(self):
    return self.__objetivo

  def getListaOrdenada(self):
    return self.__lista_ordenada.copy()

  def getVariaveis(self):
    return self.__variaveis.copy()

  def heuristica(self):
    cp = self.__capacidade

    self.__variaveis = self.__variaveis_ativadas.copy()

    if cp < 0:
      return False

    obj = self.__objetivo

    for variavel in self.__lista_ordenada:
      if(cp == 0):
        break
      if self.__modelo[variavel]['Custo'] <= cp:
        self.__variaveis.append(variavel)
        obj += self.__modelo[variavel]['Valor']
        cp -= self.__modelo[variavel]['Custo']

    return obj

  def relaxacao(self):
    cp = self.__capacidade

    if cp < 0:
      return False, False

    obj = self.__objetivo
    variavel_frac = None

    for variavel in self.__lista_ordenada:
      if(cp == 0):
        break
      if self.__modelo[variavel]['Custo'] <= cp:
        obj += self.__modelo[variavel]['Valor']
        cp -= self.__modelo[variavel]['Custo']
      else:
        variavel_frac = variavel
        obj += self.__modelo[variavel]['Valor'] * \
         (cp / self.__modelo[variavel]['Custo'])
        cp = 0

    return obj, variavel_frac

  def setLimites(self):
    self.__limite_superior, self.__variavel_frac = self.relaxacao()
    self.__limite_inferior = self.heuristica()

  def getLimiteSuperior(self):
    return self.__limite_superior

  def getLimiteInferior(self):
    return self.__limite_inferior

  def getVariavelFrac(self):
    return self.__variavel_frac

  def setGapOtimalidade(self):
    self._gap = (self.__limite_superior - self.__limite_inferior) / \
    self.__limite_superior

  def getGapOtimalidade(self):
    return self._gap

  def __eq__(self, other):
    return self.getLimiteSuperior() == other.getLimiteSuperior()

  def __lt__(self, other): # utilizado para comparação entre objetos desta classe, como no uso da heap
    return self.getLimiteSuperior() > other.getLimiteSuperior()




#Mostrando os resultados:

In [3]:
from time import time

cap = 0
pesos = []
valores = []

with open("Weakly001", 'r') as file:
    file.readline()
    file.readline()

    cap = int(file.readline().strip())

    for line in file:
        values = line.strip().split('\t')
        pesos.append(float(values[0]))
        valores.append(float((values[1])))

inicio = time()

modelo = KnapsackModel(valores, pesos, cap)
solucao = modelo.solve()

fim = time()

print(f"Tempo de execução: {fim - inicio} segundos")
print(f"Nós explorados: {modelo.getNosExplorados()}")
print(f"Função objetivo: {solucao.getLimiteSuperior()}")
print(f"Gap de otimalidade: {solucao.getGapOtimalidade()}")
vars = solucao.getVariaveis()
vars = sorted(vars, key=lambda var: int(var[1:]))
print(f"Variáveis selecionadas: {vars}")

Tempo de execução: 0.34937071800231934 segundos
Nós explorados: 1901
Função objetivo: 4886.0
Gap de otimalidade: 0.0
Variáveis selecionadas: ['x1', 'x4', 'x10', 'x13', 'x14', 'x31', 'x79', 'x103', 'x111', 'x140', 'x182', 'x219', 'x294', 'x296', 'x317', 'x329', 'x332', 'x370', 'x396', 'x471', 'x473', 'x497', 'x505', 'x512', 'x556', 'x558', 'x574', 'x591', 'x619', 'x632', 'x675', 'x687', 'x690', 'x700', 'x745', 'x752', 'x789', 'x793', 'x838', 'x842', 'x887', 'x902', 'x923', 'x931', 'x937', 'x955', 'x968', 'x979', 'x989', 'x990', 'x994']
